In [2]:
!pip install textattack

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 418.7/418.7 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 282.6/282.6 kB 26.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 373.1/373.1 kB 35.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 769.7/769.7 kB 52.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 28.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.2/125.2 kB 12.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 59.7 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.4/41.4 MB 28.8 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.5/97.5 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.8/9

In [3]:
!pip install tensorflow-text

In [4]:
import textattack
import transformers

textattack: Updating TextAttack package dependencies.
textattack: Downloading NLTK required packages.


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package omw to /usr/share/nltk_data...
[nltk_data]   Package omw is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     /usr/share/nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!
[nltk_data] Downloading package wordnet to /usr/share/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [5]:
model = transformers.AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
tokenizer = transformers.AutoTokenizer.from_pretrained("textattack/bert-base-uncased-yelp-polarity")
model_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(model, tokenizer)

In [6]:
# Construct our four components for `Attack`

from textattack.constraints.pre_transformation import RepeatModification, StopwordModification
from textattack.search_methods import GreedyWordSwapWIR
from textattack.constraints.semantics import WordEmbeddingDistance
from textattack.transformations import WordSwap

In [7]:
goal_function = textattack.goal_functions.UntargetedClassification(model_wrapper)

constraints = [

    RepeatModification(),
    StopwordModification(),
    WordEmbeddingDistance(min_cos_sim=0.7)

]

textattack: Unknown if model of class <class 'transformers.models.bert.modeling_bert.BertForSequenceClassification'> compatible with goal function <class 'textattack.goal_functions.classification.untargeted_classification.UntargetedClassification'>.
textattack: Downloading https://textattack.s3.amazonaws.com/word_embeddings/paragramcf.
100%|██████████| 481M/481M [00:18<00:00, 26.3MB/s] 
textattack: Unzipping file /root/.cache/textattack/tmp_dddqqhx.zip to /root/.cache/textattack/word_embeddings/paragramcf.
textattack: Successfully saved word_embeddings/paragramcf to cache.


In [8]:
from textattack.search_methods import GreedySearch

In [9]:
transformation = textattack.transformations.word_swaps.word_swap_embedding.WordSwapEmbedding()
search_method = textattack.search_methods.GreedySearch()

In [10]:
#attack = textattack.attack_recipes.deepwordbug_gao_2018.DeepWordBugGao2018.build(model_wrapper)
attack =  textattack.Attack(goal_function, constraints, transformation, search_method)

#datasets.load_dataset('emotion', split =['test'])
#dataset=datasets.load_dataset('sms_spam', split =['test'])
dataset = textattack.datasets.HuggingFaceDataset("yelp_polarity", dataset_columns=(["text"],'label'))
#attack =  textattack.Attack(goal_function, constraints, transformation, search_method)

Generating train split:   0%|          | 0/560000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/38000 [00:00<?, ? examples/s]

Dataset yelp_polarity downloaded and prepared to /root/.cache/huggingface/datasets/yelp_polarity/plain_text/1.0.0/14f90415c754f47cf9087eadac25823a395fef4400c7903c5897f55cfaaa6f61. Subsequent calls will reuse this data.


  0%|          | 0/2 [00:00<?, ?it/s]

textattack: Loading datasets dataset yelp_polarity, split train.


In [11]:
attack_args = textattack.AttackArgs(

    num_examples=100,
    disable_stdout=True

)
attacker = textattack.Attacker(attack, dataset, attack_args)

attacker.attack_dataset()

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



[Succeeded / Failed / Skipped / Total] 97 / 3 / 0 / 100: 100%|██████████| 100/100 [3:30:06<00:00, 126.06s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 97     |
| Number of failed attacks:     | 3      |
| Number of skipped attacks:    | 0      |
| Original accuracy:            | 100.0% |
| Accuracy under attack:        | 3.0%   |
| Attack success rate:          | 97.0%  |
| Average perturbed word %:     | 6.42%  |
| Average num. words per input: | 146.1  |
| Avg num queries:              | 4517.8 |
+-------------------------------+--------+


In [12]:
from textattack import Attacker

attacker = Attacker(attack, dataset)
attack_results=attacker.attack_dataset()

Attack(
  (search_method): GreedySearch
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  15
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.7
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): RepeatModification
    (2): StopwordModification
  (is_black_box):  True
) 



 10%|█         | 1/10 [01:46<15:55, 106.12s/it]

--------------------------------------------- Result 1 ---------------------------------------------


[Succeeded / Failed / Skipped / Total] 1 / 0 / 0 / 1:  10%|█         | 1/10 [01:46<15:58, 106.53s/it]

[[1 (100%)]] --> [[2 (86%)]]

Unfortunately, the frustration of being [[Dr]]. Goldberg's patient is a repeat of the [[experience]] I've had with so many other doctors in NYC -- good doctor, terrible [[staff]].  It seems that his staff simply never [[answers]] the phone.  It [[usually]] takes 2 hours of repeated calling to get an answer.  Who has time for that or wants to deal with it?  I have run into this problem with many other doctors and I just don't get it.  You have office workers, you have [[patients]] with medical needs, why isn't anyone [[answering]] the phone?  It's [[incomprehensible]] and not work the aggravation.  It's with regret that I feel that I have to give Dr. Goldberg 2 [[stars]].

Unfortunately, the frustration of being [[Doctor]]. Goldberg's patient is a repeat of the [[enjoying]] I've had with so many other doctors in NYC -- good doctor, terrible [[servants]].  It seems that his staff simply never [[reaction]] the phone.  It [[sometimes]] takes 2 hours of repeate

...

--------------------------------------------- Result 8 ---------------------------------------------
[[1 (100%)]] --> [[2 (68%)]]

This place is absolute [[garbage]]...  [[Half]] of the tees are not available, including all the grass tees.  It is cash only, and they sell the last bucket at 8, despite having lights.  And if you finish even a minute after 8, don't plan on getting a drink.  The vending machines are sold out (of course) and they sell drinks inside, but close the drawers at 8 on the dot.  There are [[weeds]] grown all over the place.  I noticed some sort of batting cage, but it looks like those are out of order as well.  Someone should buy this place and [[turn]] it into what it should be.

This place is absolute [[dustbin]]...  [[Averaging]] of the tees are not available, including all the grass tees.  It is cash only, and they sell the last bucket at 8, despite having lights.  And if you finish even a minute after 8, don't plan on getting a drink.  The vending machines ar

[Succeeded / Failed / Skipped / Total] 9 / 0 / 0 / 9:  90%|█████████ | 9/10 [15:17<01:41, 101.94s/it]

--------------------------------------------- Result 9 ---------------------------------------------
[[2 (100%)]] --> [[1 (97%)]]

Before I finally made it over to this [[range]] I heard the same thing from most people - it's just fine to go work on your swing. I had such a low expectation I was pleasantly [[surprised]]. \n\nIt's a fairly big range - if you are familiar with Scally's in Moon, it seems like it has almost as many tees, though its not nearly as nice a facility. \n\nThe guys in the pro shop were two of the friendlier guys I've come across at ranges or at courses. Yards were indeed marked and there are some targets to aim for, and even some hazards to aim away from. \n\nA big [[red]] flag to me was the extra charge ($3) to hit off the grass. I am no range expert, but this is the 4th one I've been to and the first I've seen of that sort of nickel and diming....\n\nPrice for the golf balls was reasonable and I do plan to be [[back]] every week until they close up in October f

[Succeeded / Failed / Skipped / Total] 10 / 0 / 0 / 10: 100%|██████████| 10/10 [15:34<00:00, 93.45s/it]

--------------------------------------------- Result 10 ---------------------------------------------
[[2 (100%)]] --> [[1 (98%)]]

I drove by yesterday to get a sneak peak.  It re-opens on July 14th and I can't wait to take my kids.  The new range looks [[amazing]].  The entire range appears to be turf, which may or many not help your game, but it looks really nice.  The tee boxes look state of the art and the club house looks like something you'll see on a newer course.  Can't [[wait]] to experience it!

I drove by yesterday to get a sneak peak.  It re-opens on July 14th and I can't wait to take my kids.  The new range looks [[striking]].  The entire range appears to be turf, which may or many not help your game, but it looks really nice.  The tee boxes look state of the art and the club house looks like something you'll see on a newer course.  Can't [[awaits]] to experience it!



+-------------------------------+--------+
| Attack Results                |        |
+----------------